this is a developer notebook for testing out red cell detection. 

If you find this notebook somewhere, ignore it! There are suite3d demos labeled "Demo-....." that are the official demos.


In [20]:
%reload_ext autoreload
%autoreload 2

import os 
from datetime import date
from matplotlib import pyplot as plt
import numpy as np

# we need to set the current path to the directory
# containing the suite3d repository, this hack should
# do the trick
os.chdir(os.path.dirname(os.path.abspath("")))

from suite3d.job import Job
from suite3d import io
from suite3d import plot_utils as plot

In [21]:
# update this to point to the demo data! 
tifs = io.get_tif_paths(r"D:\localData\suite3d\tinya_test_data_with_red")
for tif in tifs: print(tif)

D:\localData\suite3d\tinya_test_data_with_red\2025-04-09_4_TC030_2P_00001_00001.tif
D:\localData\suite3d\tinya_test_data_with_red\2025-04-09_4_TC030_2P_00001_00002.tif
D:\localData\suite3d\tinya_test_data_with_red\2025-04-09_4_TC030_2P_00001_00003.tif
D:\localData\suite3d\tinya_test_data_with_red\2025-04-09_4_TC030_2P_00001_00004.tif
D:\localData\suite3d\tinya_test_data_with_red\2025-04-09_4_TC030_2P_00001_00005.tif
D:\localData\suite3d\tinya_test_data_with_red\2025-04-09_4_TC030_2P_00001_00006.tif


In [22]:
# Set the mandatory parameters
params = {
    # volume rate
    'fs': io.get_vol_rate(tifs[0]),
    
    # planes to analyze. 0 is typically the flyback, so we exclude it here
    'planes' : np.array([0,1,2,3,4,5,6,7,8]), 
    # number of planes recorded by scanimage, including the flyback
    'n_ch_tif' : 9,
    
    # Decay time of the Ca indicator in seconds. 1.3 for GCaMP6s. This example is for GCamP8m
    'tau' : 1.3,
    'lbm' : False, 
    'num_colors' : 2, # how many color channels were recorded by scanimage
    'functional_color_channel' : 0, # which color channel is the functional one

    'process_structural_channel': True, # whether to process the structural channel
    'structural_color_channel': 1, # which color channel is the structural one
    'clear_registered_structural_data': False, # whether to clear the registered structural data after the mean volume is calculated

     # voxel size in z,y,x in microns
    'voxel_size_um' : (20, 1.5, 1.5),

    # number of files to use for the initial pass
    # usually, ~500 frames is a good rule of thumb
    # we will just use 200 here for speed
    'n_init_files' :  1,

    # 3D GPU registration - fast! 
    '3d_reg' : False,
    'gpu_reg' : True,
    'nonrigid' : True,
    
    # note : 3D CPU is not supported yet
    'subtract_crosstalk' : False, # turn off some lbm-only features
    'fuse_strips' : False, # turn off some lbm-only features    
}

In [23]:
# Create the job
job = Job(r'D:\localData\suite3d\runs','test-redcell', tifs = tifs,
          params=params, create=True, overwrite=True, verbosity = 3)

Loading job directory for test-redcell in D:\localData\suite3d\runs
      Created dir D:\localData\suite3d\runs\s3d-test-redcell\registered_fused_data
      Created dir D:\localData\suite3d\runs\s3d-test-redcell\summary
      Created dir D:\localData\suite3d\runs\s3d-test-redcell\iters
   Loading default params
      Updating param fs
      Updating param planes
      Updating param n_ch_tif
      Updating param tau
      Updating param lbm
      Updating param num_colors
      Updating param functional_color_channel
      Updating param process_structural_channel
      Updating param structural_color_channel
      Updating param clear_registered_structural_data
      Updating param voxel_size_um
      Updating param n_init_files
      Updating param 3d_reg
      Updating param gpu_reg
      Updating param nonrigid
      Updating param subtract_crosstalk
      Updating param fuse_strips
   Updated main params file


In [ ]:
job.run_init_pass()

   Saved a copy of params at D:\localData\suite3d\runs\s3d-test-redcell\summary
   Updated main params file
Launching initial pass
Saving summary to D:\localData\suite3d\runs\s3d-test-redcell\summary\summary.npy
   Loading init tifs with 9 channels
      Loading tiff 1/1: D:\localData\suite3d\tinya_test_data_with_red\2025-04-09_4_TC030_2P_00001_00004.tif
   Loaded 1 files, total 0.98 GB
   Not enough frames in loaded tifs - using 222 init frames instead
   Loaded movie with 222 frames and shape 9, 512, 512
      Enforcing positivity in mean image
   No crosstalk estimation or subtraction
   Using 2d registration


In [6]:
# If you have large tiffs, split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [7]:
# OPTIONAL: load and take a look at the reference image
summary = job.load_summary()
ref_img = summary['ref_img_3d']

structural = job.load_summary(structural=True)
ref_img_struct = structural['ref_img_3d']

# # view 1 plane at a time
# plot.show_img(ref_img[2], figsize=(3,4))
# plot.show_img(ref_img_struct[2], figsize=(3,4))

# # interactive 3D viewer
# plot.VolumeViewer(ref_img)


In [ ]:
job.register()

      Found dir D:\localData\suite3d\runs\s3d-test-redcell\registered_fused_data
      Updating self.dirs tag registered_fused_data
   Saved a copy of params at D:\localData\suite3d\runs\s3d-test-redcell\registered_fused_data
   Updated main params file
   Starting registration: 3D: False, GPU: True
Launching registration for structural channel
Will analyze 6 tifs in 6 batches
   Enforcing positivity
   Launching IO thread
         Memory at batch 0.  Total Used: 028.995 GB, Virtual Available: 034.835 GB, Virtual Used: 028.909 GB, Swap Used: 000.085 GB
Loading Batch 0 of 5
      Loading tiff 1/1: D:\localData\suite3d\tinya_test_data_with_red\2025-04-09_4_TC030_2P_00001_00001.tif
   Loaded 1 files, total 0.98 GB
   Batch 0 IO thread joined
         Memory after IO thread joinTotal Used: 029.976 GB, Virtual Available: 033.853 GB, Virtual Used: 029.891 GB, Swap Used: 000.085 GB
         Memory after movie copied from threadTotal Used: 030.956 GB, Virtual Available: 032.873 GB, Virtual Use

KeyboardInterrupt: 

In [9]:
import napari
import tifffile as tf

mov_green = job.get_registered_movie(key="registered_fused_data", filename_filter="fused", structural=False)[:, :200].compute()
mov_red = job.get_registered_movie(key="registered_fused_data", filename_filter="fused", structural=True)[:, :200].compute()

# Create the viewer and set human-readable axis labels
viewer = napari.Viewer()
viewer.dims.axis_labels = ['Plane', 'Time', 'Y', 'X']

# Add each movie as a separate layer with additive blending
viewer.add_image(
    mov_red,
    name='Red channel',
    colormap='red',
    blending='additive',
    visible=True
)
viewer.add_image(
    mov_green,
    name='Green channel',
    colormap='green',
    blending='additive',
    visible=True
)

napari.run()

In [ ]:
ref_img_3d_structural = job.get_registered_movie("registered_fused_data", "fused", structural=True)

In [13]:
ref_img_3d = job.load_file("ref_img_3d", "summary")
ref_img_3d_structural = job.load_file("ref_img_3d_structural", "summary")

print(ref_img_3d.shape, ref_img_3d_structural.shape)

      Loading from D:\localData\suite3d\runs\s3d-test-redcell\summary\ref_img_3d.npy
      Loading from D:\localData\suite3d\runs\s3d-test-redcell\summary\ref_img_3d_structural.npy
(9, 517, 516) (9, 518, 519)


In [ ]:
corr_map = job.calculate_corr_map()

   Created dir D:\localData\suite3d\runs\s3d-test-redcell\corrmap with tag corrmap
      Updating self.dirs tag corrmap
   Created dir D:\localData\suite3d\runs\s3d-test-redcell\mov_sub with tag mov_sub
      Updating self.dirs tag mov_sub
   Updated detection_timebin to 23 based on framerate and tau
   Saved a copy of params at D:\localData\suite3d\runs\s3d-test-redcell\corrmap
   Updated main params file
   Computing correlation map of movie with 1333 frames, volume shape: 9, 517, 516
      Running batch 1 of 2
      Binning with timebin of size 23


C:\Users\Andrew\AppData\Local\Temp\ipykernel_21972\472069948.py:1: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  corr_map = job.calculate_corr_map()


         Timer batch_timebin completed in 0.011 sec
         Timer prep completed in 2.290 sec
         Timer batch_setup completed in 0.085 sec
         Timer batch_edgecrop completed in 0.011 sec
         Timer accum_meanmeax completed in 0.064 sec
         Timer batch_rolling_mean_filt completed in 0.050 sec
         Timer batch_accum_sdmov completed in 0.176 sec
         Timer batch_norm_sdmov completed in 0.037 sec
         Loading movie of size (34, 9, 517, 516) into shared memory
         Timer dtu_shmem completed in 0.230 sec
         Subtracting neuropil and applying cell filters
         Timer dtu_npsub_conv3d completed in 2.495 sec
         Reducing filtered movie to compute correlation map
         Timer dtu_vmap completed in 0.368 sec
         Timer dtu_cleanup completed in 0.133 sec
         Timer batch_filt_reduce completed in 3.277 sec
         Timer batch_accum_vmap completed in 0.023 sec
         Timer batch completed in 3.844 sec
         Timer save completed in 0.50

C:\Users\Andrew\AppData\Local\Temp\ipykernel_21972\472069948.py:1: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  corr_map = job.calculate_corr_map()


         Timer prep completed in 2.603 sec
         Timer batch_setup completed in 0.041 sec
         Timer batch_edgecrop completed in 0.005 sec
         Timer accum_meanmeax completed in 0.081 sec
         Timer batch_rolling_mean_filt completed in 0.050 sec
         Timer batch_accum_sdmov completed in 0.134 sec
         Timer batch_norm_sdmov completed in 0.069 sec
         Loading movie of size (23, 9, 517, 516) into shared memory
         Timer dtu_shmem completed in 0.231 sec
         Subtracting neuropil and applying cell filters


KeyboardInterrupt: 

In [60]:
res = job.load_corr_map_results()
vmap = res['vmap']

In [61]:
job.params['patch_size_xy'] = (550, 550)
# for speed, only segment a single patch
job.segment_rois()

   Created dir D:\localData\suite3d\runs\s3d-test-redcell\segmentation with tag segmentation
      Updating self.dirs tag segmentation
   Saved a copy of params at D:\localData\suite3d\runs\s3d-test-redcell\segmentation
   Updated main params file
   Created dir D:\localData\suite3d\runs\s3d-test-redcell\rois with tag rois
      Updating self.dirs tag rois
   Saving results to D:\localData\suite3d\runs\s3d-test-redcell\segmentation and D:\localData\suite3d\runs\s3d-test-redcell\rois 
dict_keys(['max_img', 'mean_img', 'vmap', 'vmap_raw', 'all_params'])
   Detecting from patch 1 / 1
   Created dir D:\localData\suite3d\runs\s3d-test-redcell\segmentation\patch-0000 with tag segmentation-patch-0000
         Loading 0.25 GB movie to memory, shape: (57, 9, 517, 516) 
         Loaded
         Loading movie patch to shared memory
         Loaded
      Starting extraction with peak_thresh: 0.100 and Th2: 5.000
         Iter 0000: running 16 ROIs in parallel
         Added cell 1 at 04, 409, 050,

'D:\\localData\\suite3d\\runs\\s3d-test-redcell\\rois'

In [62]:
job.compute_npil_masks()
traces = job.extract_and_deconvolve()

   Updated main params file
   Movie shape: (9, 1333, 517, 516)
2350
   Extracting 2350 valid cells, and saving cell flags to D:\localData\suite3d\runs\s3d-test-redcell\rois\iscell_extracted.npy
   Extracting activity
         Will extract in 3 batches of 500
   Saving intermediate results to D:\localData\suite3d\runs\s3d-test-redcell\rois
   Deconvolving
   Saving to D:\localData\suite3d\runs\s3d-test-redcell\rois


In [68]:
results_path = r'D:\localData\suite3d\results\test-redcell'
job.export_results(results_path, result_dir_name='rois')

   Created dir D:\localData\suite3d\results\test-redcell\s3d-results-test-redcell to export results
      Loading from D:\localData\suite3d\runs\s3d-test-redcell\rois\stats_small.npy
      Loading from D:\localData\suite3d\runs\s3d-test-redcell\rois\info.npy
      Loading from D:\localData\suite3d\runs\s3d-test-redcell\rois\F.npy
      Loading from D:\localData\suite3d\runs\s3d-test-redcell\rois\spks.npy
      Loading from D:\localData\suite3d\runs\s3d-test-redcell\rois\Fneu.npy
      Loading from D:\localData\suite3d\runs\s3d-test-redcell\rois\iscell.npy
      Loading from D:\localData\suite3d\runs\s3d-test-redcell\summary\ref_img_3d_structural.npy
      Loading from D:\localData\suite3d\runs\s3d-test-redcell\summary\ref_img_3d.npy
      Overwriting existing D:\localData\suite3d\results\test-redcell\s3d-results-test-redcell\s3d-params.npy
      Overwriting existing D:\localData\suite3d\results\test-redcell\s3d-results-test-redcell\frames.npy
      Overwriting existing D:\localData\sui

To take a look at the outputs in napari, navigate to the suite3d directory in a command shell and run the following:
```
python curation.py curation --output_dir /path/to/output/rois
```

In [2]:
# Code for developing red cell analysis for 3D ROIs

from pathlib import Path
import numpy as np
results_path = Path(r"D:\localData\suite3d\results\test-redcell\s3d-results-test-redcell")

stats = np.load(results_path / "stats.npy", allow_pickle=True)
iscell = np.load(results_path / "iscell.npy")

In [6]:
stats[0]["lam"]

array([0.0096192 , 0.0008921 , 0.00102622, ..., 0.01933373, 0.0056079 ,
       0.01602068])

In [86]:
idxcells = np.where(iscell[:, 0])[0]
s = stats[idxcells[0]]